In [3]:
import numpy as np

# Variables

Helium viscosity from 2K to 5K: https://royalsocietypublishing.org/doi/10.1098/rspa.1950.0181

Gas helium viscosity: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392

Stainless steel thermal conductivity: https://thermtest.com/thermal-conductivity-of-steel

In [190]:
OD_above_50K = 0.25*0.0254 #m
OD_below_50K = 0.125*0.0254 #m

thickness_above_50K = (0.25-0.18)*0.0254/2 #m, capillary wall thickness
thickness_below_50K = (0.125-0.085)*0.0254/2 #m, capillary wall thickness

length_above_50K = 10*0.0254 #m
length_below_50K = 4*20*0.0254 #m

pressure_warm_end_condensation = 1*10**5 #Pa
pressure_cold_end_condensation = 0.25*10**5 #Pa, approximation
pressure_50K_condensation = 0.5*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

pressure_warm_end_evaporation = 0.01*10**5 #Pa
pressure_cold_end_evaporation = 0.25*10**5 #Pa, approximation
pressure_50K_evaporation = 0.05*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

viscosity_gas = 0.0196*10**(-3) #Pa*s, https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392
viscosity_liquid = 0.03*10**(-3) #Pa*s, https://royalsocietypublishing.org/doi/epdf/10.1098/rspa.1950.0181

stainless_steel_thermal_conductivity = 15*10**(-3) #W/K, https://thermtest.com/thermal-conductivity-of-steel

helium_volume = 0.1 #m^3, approximation

Poiseuille's Law: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/

In [13]:
#radius is the inner radius
def getTubeVolmeFlowRate(viscosity, startPressure, endPressure, length, radius):
    R = 8*viscosity*length/(np.pi*(radius**4))
    return (startPressure-endPressure)/R

Fourier's law of heat conduction for cylinders, page 7 on https://www.sfu.ca/~mbahrami/ENSC%20388/Notes/Staedy%20Conduction%20Heat%20Transfer.pdf

In [177]:
def getTubeHeatFlowRate(outerTemperature, innerTemperature, outerRadius, innerRadius, length, thermalConductivity):
    R = np.log(outerRadius/innerRadius)/(2*np.pi*thermalConductivity*length)
    return (innerTemperature - outerTemperature)/R

In [180]:
#1D linear temperature gradient
def getHeatLoad(outerTemperature, innerTemperature, outerRadius, innerRadius, length, thermalConductivity, time):
    step_num = 100
    temp_difference = innerTemperature - outerTemperature
    heat_load = 0 #initialization
    for i in range(step_num):
        innerTemperature_current = innerTemperature-i*temp_difference/step_num
        heatFlowRate = getTubeHeatFlowRate(outerTemperature, innerTemperature_current, outerRadius, innerRadius, length/step_num, thermalConductivity)
        heat_load = heat_load+heatFlowRate*time
        
    return heat_load #in W

# Gas throughput

## Condensation time

In [191]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
condensation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_warm_end_condensation,pressure_50K_condensation,length_above_50K,inner_radius_above_50K)
condensation_time_above_50K = helium_volume/condensation_volume_flow_rate_above_50K
print(condensation_volume_flow_rate_above_50K)
print(condensation_time_above_50K) #in seconds

0.10770726704988587
0.9284424601887248


In [192]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
condensation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_condensation,pressure_cold_end_condensation,length_below_50K,inner_radius_below_50K)
condensation_time_below_50K = helium_volume/condensation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(condensation_time_below_50K) #in seconds

0.005355878159813628
298.73719159729285


The condensation time will be limited by the section below the 50K plate, which is 298 seconds.

## Evaporation time

In [193]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
evaporation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_evaporation,pressure_warm_end_evaporation,length_above_50K,inner_radius_above_50K)
evaporation_time_above_50K = helium_volume/evaporation_volume_flow_rate_above_50K
print(volume_flow_rate_above_50K)
print(evaporation_time_above_50K) #in seconds

1.723316272798174
11.605530752359059


In [194]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
evaporation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_cold_end_evaporation,pressure_50K_evaporation,length_below_50K,inner_radius_below_50K)
evaporation_time_below_50K = helium_volume/evaporation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(evaporation_time_below_50K) #in seconds

0.005355878159813628
373.42148949661606


The evaporation time will be limited by the section below the 50K plate, which is 373 seconds.

# Heat Load (unit in W)

In [195]:
heat_flow_rate_above_50K = getHeatLoad(42, 300, OD_above_50K/2, inner_radius_above_50K, length_above_50K, stainless_steel_thermal_conductivity,evaporation_time_above_50K)
print(heat_flow_rate_above_50K)

110.18949284769614


In [196]:
heat_flow_rate_4K_50K = getHeatLoad(3, 42, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_flow_rate_4K_50K)

913.025151078126


In [197]:
heat_flow_rate_still_4K = getHeatLoad(0.9, 3, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_load_still_4K)

3.9891684912618754


In [198]:
heat_flow_rate_cold_still = getHeatLoad(0.08, 0.9, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_load_cold_still)

1.5576753156355894


In [199]:
heat_flow_rate_MXC_cold = getHeatLoad(0.02,0.08, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_load_MXC_cold)

0.11397624260748214


In [200]:
#total heat load in W
print(sum([heat_load_above_50K,heat_load_4K_50K,heat_load_still_4K,heat_load_cold_still,heat_load_MXC_cold]))

84.35328819475714


# Alternative radius choice for the section below 50 K

In [201]:
OD_below_50K = 0.0625*0.0254 #m
thickness_below_50K = (0.0625-0.0425)*0.0254/2 #m, capillary wall thickness

# Gas throughput

## Condensation time

In [202]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
condensation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_warm_end_condensation,pressure_50K_condensation,length_above_50K,inner_radius_above_50K)
condensation_time_above_50K = helium_volume/condensation_volume_flow_rate_above_50K
print(condensation_volume_flow_rate_above_50K)
print(condensation_time_above_50K) #in seconds

0.10770726704988587
0.9284424601887248


In [203]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
condensation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_condensation,pressure_cold_end_condensation,length_below_50K,inner_radius_below_50K)
condensation_time_below_50K = helium_volume/condensation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(condensation_time_below_50K) #in seconds

0.005355878159813628
4779.795065556686


The condensation time will be limited by the section below the 50K plate, which is 4779 seconds.

## Evaporation time

In [204]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
evaporation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_evaporation,pressure_warm_end_evaporation,length_above_50K,inner_radius_above_50K)
evaporation_time_above_50K = helium_volume/evaporation_volume_flow_rate_above_50K
print(volume_flow_rate_above_50K)
print(evaporation_time_above_50K) #in seconds

1.723316272798174
11.605530752359059


In [205]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
evaporation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_cold_end_evaporation,pressure_50K_evaporation,length_below_50K,inner_radius_below_50K)
evaporation_time_below_50K = helium_volume/evaporation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(evaporation_time_below_50K) #in seconds

0.005355878159813628
5974.743831945857


The evaporation time will be limited by the section below the 50K plate, which is 5974 seconds.

# Heat Load

In [213]:
heat_flow_rate_above_50K = getHeatLoad(42, 300, OD_above_50K/2, inner_radius_above_50K, length_above_50K, stainless_steel_thermal_conductivity,evaporation_time_above_50K)
print(heat_flow_rate_above_50K) #W

110.18949284769614


In [214]:
heat_flow_rate_4K_50K = getHeatLoad(3, 42, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_flow_rate_4K_50K) #W

14608.402417250016


In [215]:
heat_flow_rate_still_4K = getHeatLoad(0.9, 3, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_flow_rate_still_4K)

786.6062840057701


In [216]:
heat_flow_rate_cold_still = getHeatLoad(0.08, 0.9, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_flow_rate_cold_still)

307.15102518320543


In [217]:
heat_flow_rate_MXC_cold = getHeatLoad(0.02,0.08, OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity,evaporation_time_below_50K)
print(heat_flow_rate_MXC_cold)

22.474465257307717


In [218]:
#total heat load in W
print(sum([heat_flow_rate_above_50K,heat_flow_rate_4K_50K,heat_flow_rate_still_4K,heat_flow_rate_cold_still,heat_flow_rate_MXC_cold]))

15834.823684543995
